In [62]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re 
import time


## SCRAPING WEBSITE 1 (BAYUT WEBSITE)

In [10]:
# 1. Define the base URL of the website
base_url = "https://www.bayut.om/en/oman/properties-for-sale/"
# Initialize the current page URL to the base URL
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"}
current_page_url = base_url

In [11]:
all_Properties_sales = {
    'prop_price' : [], 'prop_title': [],
         'prop_location': [], 'prop_size': [],
        'number_room': []  
}
# proxy={"https": "https://192.168.1.1:443"}
r=requests.get(base_url, headers=headers)

print(f"Starting to scrape from: {base_url}")
print(r.status_code)

Starting to scrape from: https://www.bayut.om/en/oman/properties-for-sale/
200


In [12]:
r.text

'<!DOCTYPE html><html lang="en" dir="ltr" itemscope="" recommenderthreelistingsondetailpages="recommended_only" itemType="http://schema.org/WebPage"><head><meta charSet="UTF-8"/><meta name="viewport" content="width=device-width, initial-scale=1.0, user-scalable=0"/><link rel="dns-prefetch" href="http://LL8IZ711CS-dsn.algolia.net"/><link rel="dns-prefetch" href="https://www.googletagmanager.com"/><link rel="dns-prefetch" href="https://www.google-analytics.com"/><link rel="dns-prefetch" href="https://images.bayut.com"/><link rel="apple-touch-icon" sizes="180x180" href="/assets/apple-touch-icon.7919a66ff6f58dbfb1d2e33cb19b5a80.png"/><link rel="icon" type="image/png" sizes="16x16" href="/assets/favicon-16x16.0e9b01e26e8e124b4b030829ed6e5bfb.png"/><link rel="icon" type="image/png" sizes="32x32" href="/assets/favicon-32x32.20ce04f8a637e94fd0dc5ca4eeab1dce.png"/><link rel="manifest" href="/assets/9747ba91762d335f81891519d885a64f.json" crossorigin="use-credentials"/><link rel="mask-icon" href=

In [13]:
r.status_code

200

In [14]:
# Loop through pages until no 'next' button is found
while True:
    print(f"Scraping page: {current_page_url}")
    try:
        response = requests.get(current_page_url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {current_page_url}: {e}")
        break # Exit loop on request error
    soup = BeautifulSoup(response.content, 'html.parser')
    property_info = soup.find_all('div', class_ ='_475e888a')

    for prop in property_info:
        prop_price = prop.find('span', class_ = 'dc381b54')
        if prop_price:
            prop_prices = prop_price.text.strip()
            all_Properties_sales['prop_price'].append(prop_prices)
        else:
            all_Properties_sales['prop_price'].append('not available')
    
    
        prop_title = prop.find('span', class_ = '_19e94678 e0abc2de')
        if prop_title:
            prop_title = prop_title.text.strip()
            all_Properties_sales['prop_title'].append(prop_title)
        else:
            all_Properties_sales['prop_title'].append('not available')
            
        prop_location = prop.find('h3', class_ = '_4402bd70')
        if prop_location:
            prop_location = prop_location.text.strip()
            all_Properties_sales['prop_location'].append(prop_location)
        else:
            all_Properties_sales['prop_location'].append('not available')

        number_room = prop.find('span', attrs={'aria-label': 'Beds'})
        if number_room:
            number_room = number_room.text.strip()
            all_Properties_sales['number_room'].append(number_room)
        else:
            all_Properties_sales['number_room'].append('not available')
   
        # prop_size = prop.find('span',class_ = '_140e6903')
        prop_size = prop.find('h4', class_="cfac7e1b _85ddb82f")
        if prop_size:
            prop_size = prop_size.text
            all_Properties_sales['prop_size'].append(prop_size)
        else:
            all_Properties_sales['prop_size'].append('not available')
            
        next_page_anchor = soup.find('a', class_='_95dd93c1', attrs={'title': 'Next'}, href=True)
        if next_page_anchor:
            next_page_relative_url = next_page_anchor['href']
            new_url = requests.compat.urljoin(base_url, next_page_relative_url)

    # Prevent infinite loop if same page repeats
    if new_url == current_page_url:
        print("no next page")
        break
    else:
        current_page_url = new_url
        time.sleep(1)


Scraping page: https://www.bayut.om/en/oman/properties-for-sale/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-2/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-3/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-4/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-5/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-6/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-7/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-8/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-9/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-10/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-11/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-12/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/page-13/
Scraping page: https://www.bayut.om/en/oman/properties-for-sale/pag

### SAVE DATA TO CSV FILE 

In [16]:
df_1= pd.DataFrame(all_Properties_sales)
df_1.to_csv("bayut_detailed.csv", index=False)
df_1

,prop_price,prop_title,prop_location,prop_size,number_room
0,"1,200",Villa,"Al Buraimi, Al Buraimi","1,500 Sq. M.",5
1,"1,500",Villa,"Qantab, Muscat","1,500 Sq. M.",5
2,"1,200",Apartment,"Mahut, Al Wusta","1,000 Sq. M.",3
3,"1,500",Villa,"Al Sifah, Muscat","1,000 Sq. M.",5
4,"1,000",Villa,"Hamra, Al Dakhiliya","1,000 Sq. M.",4
...,...,...,...,...,...
1804,"1,000",Villa,"Haima, Al Wusta","1,000 Sq. M.",4
1805,600,Apartment,"Mahut, Al Wusta",10 Sq. M.,1
1806,"5,800",Residential Land,"Nizwa, Al Dakhiliya",600 Sq. M.,not available
1807,"491,410",Villa,"Yiti, Muscat",384 Sq. M.,4


## SCRAPING WEBSITE 2 (OPEN SOOQ WEBSITE)

In [2]:
base_url2 = "https://om.opensooq.com/en/property/property-for-sale"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/114.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "keep-alive",
    "Referer": "https://www.google.com"
}
current_page_url2 = base_url2

In [3]:
data = {
    'prop_price' : [], 'prop_title': [],
         'prop_location': [], 'prop_size': [],
        }

In [4]:
while True:
    print(f"Scraping page: {current_page_url2}")
    try:
        response = requests.get(current_page_url2)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching page {current_page_url2}: {e}")
        break # Exit loop on request error
    soup = BeautifulSoup(response.content, 'html.parser')
    property_info = soup.find_all('div', class_ ='postDet flex flex-1 flexWrap flexSpaceBetween flexDirectionColumn overflowHidden ripple')
    #print(property_info)
    for prop in property_info:
        prop_title = prop.find('div', class_ = 'flex flexSpaceBetween gap-10')
        if prop_title:
            prop_title = prop_title.text.strip()
            data['prop_title'].append(prop_title)
        else:
            data['prop_title'].append('not available')

        prop_price = prop.find('div', class_ = 'priceColor bold alignSelfCenter font-18 ms-auto')
        if prop_price:
            prop_price = prop_price.text.strip()
            data['prop_price'].append(prop_price)
        else:
            data['prop_price'].append('not available')
            
        prop_location = prop.find('div' , class_ = 'flex alignItems gap-5 darkGrayColor')
        if prop_location:
            prop_location = prop_location.text.strip()
            data['prop_location'].append(prop_location)
        else:
            data['prop_location'].append('not availble')
            
        prop_size = prop.find('div', class_ = 'flex flexSpaceBetween gap-10')
        if prop_size:
            prop_size = prop_size.text.strip()
            data['prop_size'].append(prop_size)
        else:
            data['prop_size'].append('not availble')
        next_page_anchor = soup.find('a', class_='blueHoverBg border pagerItem flex alignItems justifyContent radius-8 ripple', attrs={'title':"Go to next page"}, href=True)
        if next_page_anchor:
            next_page_relative_url = next_page_anchor['href']
            new_url = requests.compat.urljoin(base_url2, next_page_relative_url)

    # Prevent infinite loop if same page repeats
    if new_url == current_page_url2:
        print("no next page")
        break
    else:
        current_page_url2 = new_url
        time.sleep(5)
# if all_Properties_sales:  # only write if there's data
#     df_1= pd.DataFrame(all_Properties_sales)
#     df_1.to_csv("bayut_detailed.csv", index=False)
# else:
#     print("No data collected. CSV will not be created.")
    

Scraping page: https://om.opensooq.com/en/property/property-for-sale
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=2
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=3
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=4
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=5
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=6
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=7
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=8
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=9
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=10
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=11
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=12
Scraping page: https://om.opensooq.com/en/property/property-for-sale?page=13
Scraping page: 

### SAVE DATA TO CSV FILE

In [7]:
df_2= pd.DataFrame(data)
df_2.to_csv("souq_detailed.csv", index=False)

In [8]:
df_2

,prop_price,prop_title,prop_location,prop_size
0,"79,000 OMR",72 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Mouj",72 m2 Studio Apartments for Sale in Muscat Al ...
1,"8,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
2,"67,000 OMR",416 m2 More than 6 bedrooms Villa for Sale in ...,"Muscat, Amerat",416 m2 More than 6 bedrooms Villa for Sale in ...
3,"6,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
4,"43,000 OMR",92 m2 3 Bedrooms Apartments for Sale in Muscat...,"Muscat, Al Mawaleh",92 m2 3 Bedrooms Apartments for Sale in Muscat...
...,...,...,...,...
8262,"42,500 OMR",116 m2 2 Bedrooms Apartments for Sale in Musca...,"Muscat, Al Khoud",116 m2 2 Bedrooms Apartments for Sale in Musca...
8263,15 OMR,120 m2 2 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",120 m2 2 Bedrooms Apartments for Rent in Musca...
8264,"1,200 OMR",80 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Khoud",80 m2 Studio Apartments for Sale in Muscat Al ...
8265,200 OMR,100 m2 3 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",100 m2 3 Bedrooms Apartments for Rent in Musca...


## Read Bayut Data, Clean It, and Apply Feature Engineering


In [63]:
df_1 = pd.read_csv("bayut_detailed.csv")
df_1

,prop_price,prop_title,prop_location,prop_size,number_room
0,"1,200",Villa,"Al Buraimi, Al Buraimi","1,500 Sq. M.",5
1,"1,500",Villa,"Qantab, Muscat","1,500 Sq. M.",5
2,"1,200",Apartment,"Mahut, Al Wusta","1,000 Sq. M.",3
3,"1,500",Villa,"Al Sifah, Muscat","1,000 Sq. M.",5
4,"1,000",Villa,"Hamra, Al Dakhiliya","1,000 Sq. M.",4
...,...,...,...,...,...
1804,"1,000",Villa,"Haima, Al Wusta","1,000 Sq. M.",4
1805,600,Apartment,"Mahut, Al Wusta",10 Sq. M.,1
1806,"5,800",Residential Land,"Nizwa, Al Dakhiliya",600 Sq. M.,not available
1807,"491,410",Villa,"Yiti, Muscat",384 Sq. M.,4


In [64]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1809 entries, 0 to 1808
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   prop_price     1809 non-null   object
 1   prop_title     1809 non-null   object
 2   prop_location  1809 non-null   object
 3   prop_size      1809 non-null   object
 4   number_room    1809 non-null   object
dtypes: object(5)
memory usage: 70.8+ KB


In [65]:
df_1.isnull().sum()

prop_price       0
prop_title       0
prop_location    0
prop_size        0
number_room      0
dtype: int64

In [66]:
df_1['prop_price']

0         1,200
1         1,500
2         1,200
3         1,500
4         1,000
         ...   
1804      1,000
1805        600
1806      5,800
1807    491,410
1808      7,000
Name: prop_price, Length: 1809, dtype: object

In [67]:
df_1[df_1['prop_price'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room


In [68]:
df_1['prop_title']

0                  Villa
1                  Villa
2              Apartment
3                  Villa
4                  Villa
              ...       
1804               Villa
1805           Apartment
1806    Residential Land
1807               Villa
1808    Residential Land
Name: prop_title, Length: 1809, dtype: object

In [69]:
df_1[df_1['prop_title'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room


In [70]:
df_1['prop_location']

0       Al Buraimi, Al Buraimi
1               Qantab, Muscat
2              Mahut, Al Wusta
3             Al Sifah, Muscat
4          Hamra, Al Dakhiliya
                 ...          
1804           Haima, Al Wusta
1805           Mahut, Al Wusta
1806       Nizwa, Al Dakhiliya
1807              Yiti, Muscat
1808         Al Amarat, Muscat
Name: prop_location, Length: 1809, dtype: object

In [71]:
df_1[df_1['prop_location'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room


In [72]:
df_1['prop_size']

0       1,500 Sq. M.
1       1,500 Sq. M.
2       1,000 Sq. M.
3       1,000 Sq. M.
4       1,000 Sq. M.
            ...     
1804    1,000 Sq. M.
1805       10 Sq. M.
1806      600 Sq. M.
1807      384 Sq. M.
1808      645 Sq. M.
Name: prop_size, Length: 1809, dtype: object

In [73]:
df_1[df_1['prop_size'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room


In [74]:
df_1['number_room']

0                   5
1                   5
2                   3
3                   5
4                   4
            ...      
1804                4
1805                1
1806    not available
1807                4
1808    not available
Name: number_room, Length: 1809, dtype: object

In [75]:
df_1[df_1['number_room'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room
48,"12,300",Residential Land,"Thumrait, Dhofar",586 Sq. M.,not available
62,"6,200",Residential Land,"Al Amarat, Muscat","1,000 Sq. M.",not available
71,"31,000",Residential Land,"Mirbat, Dhofar",600 Sq. M.,not available
89,"34,000",Residential Land,"Barka, Al Batinah",600 Sq. M.,not available
96,"5,500",Residential Land,"Nizwa, Al Dakhiliya",600 Sq. M.,not available
...,...,...,...,...,...
1800,"6,500",Residential Land,"Ibri, Al Dhahirah",840 Sq. M.,not available
1802,"59,000",Residential Land,"Nizwa, Al Dakhiliya",531 Sq. M.,not available
1803,"6,500",Residential Land,"Ibri, Al Dhahirah",700 Sq. M.,not available
1806,"5,800",Residential Land,"Nizwa, Al Dakhiliya",600 Sq. M.,not available


In [76]:
df_1.drop(df_1[df_1['number_room'] == 'not available'].index, inplace=True)
df_1[df_1['number_room'] == 'not available']

,prop_price,prop_title,prop_location,prop_size,number_room


In [77]:
df_1

,prop_price,prop_title,prop_location,prop_size,number_room
0,"1,200",Villa,"Al Buraimi, Al Buraimi","1,500 Sq. M.",5
1,"1,500",Villa,"Qantab, Muscat","1,500 Sq. M.",5
2,"1,200",Apartment,"Mahut, Al Wusta","1,000 Sq. M.",3
3,"1,500",Villa,"Al Sifah, Muscat","1,000 Sq. M.",5
4,"1,000",Villa,"Hamra, Al Dakhiliya","1,000 Sq. M.",4
...,...,...,...,...,...
1789,"45,000",Apartment,"Qurum, Muscat",75 Sq. M.,1
1801,"75,000",Villa,"Al Seeb, Muscat",221 Sq. M.,3
1804,"1,000",Villa,"Haima, Al Wusta","1,000 Sq. M.",4
1805,600,Apartment,"Mahut, Al Wusta",10 Sq. M.,1


In [78]:
#removing commas and extract number + unit
df_1['prop_size_cleaned'] = df_1['prop_size'].str.replace(',', '', regex=False)

# Extract number and unit into two new columns
df_1[['prop_size', 'prop_size_units']] = df_1['prop_size_cleaned'].str.extract(r'(\d+\.?\d*)\s*(.*)', expand=True)

# Convert size column to numeric
df_1['prop_size'] = pd.to_numeric(df_1['prop_size'], errors='coerce')

# Drop the temporary cleaned column
df_1.drop(columns=['prop_size_cleaned'], inplace=True)

In [79]:
col_to_move = df_1.pop('prop_size_units')  # temporarily remove the column
df_1.insert(loc=df_1.columns.get_loc('prop_size') + 1, column='prop_size_units', value=col_to_move)

In [80]:
df_1

,prop_price,prop_title,prop_location,prop_size,prop_size_units,number_room
0,"1,200",Villa,"Al Buraimi, Al Buraimi",1500,Sq. M.,5
1,"1,500",Villa,"Qantab, Muscat",1500,Sq. M.,5
2,"1,200",Apartment,"Mahut, Al Wusta",1000,Sq. M.,3
3,"1,500",Villa,"Al Sifah, Muscat",1000,Sq. M.,5
4,"1,000",Villa,"Hamra, Al Dakhiliya",1000,Sq. M.,4
...,...,...,...,...,...,...
1789,"45,000",Apartment,"Qurum, Muscat",75,Sq. M.,1
1801,"75,000",Villa,"Al Seeb, Muscat",221,Sq. M.,3
1804,"1,000",Villa,"Haima, Al Wusta",1000,Sq. M.,4
1805,600,Apartment,"Mahut, Al Wusta",10,Sq. M.,1


In [81]:
df_1.insert(loc=df_1.columns.get_loc('prop_price') + 1, column='prop_price_unit', value='OMR')
df_1

,prop_price,prop_price_unit,prop_title,prop_location,prop_size,prop_size_units,number_room
0,"1,200",OMR,Villa,"Al Buraimi, Al Buraimi",1500,Sq. M.,5
1,"1,500",OMR,Villa,"Qantab, Muscat",1500,Sq. M.,5
2,"1,200",OMR,Apartment,"Mahut, Al Wusta",1000,Sq. M.,3
3,"1,500",OMR,Villa,"Al Sifah, Muscat",1000,Sq. M.,5
4,"1,000",OMR,Villa,"Hamra, Al Dakhiliya",1000,Sq. M.,4
...,...,...,...,...,...,...,...
1789,"45,000",OMR,Apartment,"Qurum, Muscat",75,Sq. M.,1
1801,"75,000",OMR,Villa,"Al Seeb, Muscat",221,Sq. M.,3
1804,"1,000",OMR,Villa,"Haima, Al Wusta",1000,Sq. M.,4
1805,600,OMR,Apartment,"Mahut, Al Wusta",10,Sq. M.,1


In [82]:
# Split  into 'state' and 'governorate'
df_1[['state', 'governorate']] = df_1['prop_location'].str.split(',', n=1, expand=True)

# Remove leading/trailing whitespace
df_1['state'] = df_1['state'].str.strip()
df_1['governorate'] = df_1['governorate'].str.strip()

# Drop the original column
df_1.drop(columns=['prop_location'], inplace=True)
df_1

,prop_price,prop_price_unit,prop_title,prop_size,prop_size_units,number_room,state,governorate
0,"1,200",OMR,Villa,1500,Sq. M.,5,Al Buraimi,Al Buraimi
1,"1,500",OMR,Villa,1500,Sq. M.,5,Qantab,Muscat
2,"1,200",OMR,Apartment,1000,Sq. M.,3,Mahut,Al Wusta
3,"1,500",OMR,Villa,1000,Sq. M.,5,Al Sifah,Muscat
4,"1,000",OMR,Villa,1000,Sq. M.,4,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
1789,"45,000",OMR,Apartment,75,Sq. M.,1,Qurum,Muscat
1801,"75,000",OMR,Villa,221,Sq. M.,3,Al Seeb,Muscat
1804,"1,000",OMR,Villa,1000,Sq. M.,4,Haima,Al Wusta
1805,600,OMR,Apartment,10,Sq. M.,1,Mahut,Al Wusta


In [83]:
df_1.dtypes

prop_price         object
prop_price_unit    object
prop_title         object
prop_size           int64
prop_size_units    object
number_room        object
state              object
governorate        object
dtype: object

In [84]:
df_1['prop_price'] = df_1['prop_price'].str.replace(',', '', regex=False)
df_1['prop_price'] = df_1['prop_price'].astype(int)
df_1['number_room'] = df_1['number_room'].astype(int)
df_1

,prop_price,prop_price_unit,prop_title,prop_size,prop_size_units,number_room,state,governorate
0,1200,OMR,Villa,1500,Sq. M.,5,Al Buraimi,Al Buraimi
1,1500,OMR,Villa,1500,Sq. M.,5,Qantab,Muscat
2,1200,OMR,Apartment,1000,Sq. M.,3,Mahut,Al Wusta
3,1500,OMR,Villa,1000,Sq. M.,5,Al Sifah,Muscat
4,1000,OMR,Villa,1000,Sq. M.,4,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
1789,45000,OMR,Apartment,75,Sq. M.,1,Qurum,Muscat
1801,75000,OMR,Villa,221,Sq. M.,3,Al Seeb,Muscat
1804,1000,OMR,Villa,1000,Sq. M.,4,Haima,Al Wusta
1805,600,OMR,Apartment,10,Sq. M.,1,Mahut,Al Wusta


In [85]:
df_1.dtypes

prop_price          int64
prop_price_unit    object
prop_title         object
prop_size           int64
prop_size_units    object
number_room         int64
state              object
governorate        object
dtype: object

In [86]:
first = df_1.pop('prop_title')      
df_1.insert(0, 'prop_title', first)
df_1

,prop_title,prop_price,prop_price_unit,prop_size,prop_size_units,number_room,state,governorate
0,Villa,1200,OMR,1500,Sq. M.,5,Al Buraimi,Al Buraimi
1,Villa,1500,OMR,1500,Sq. M.,5,Qantab,Muscat
2,Apartment,1200,OMR,1000,Sq. M.,3,Mahut,Al Wusta
3,Villa,1500,OMR,1000,Sq. M.,5,Al Sifah,Muscat
4,Villa,1000,OMR,1000,Sq. M.,4,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
1789,Apartment,45000,OMR,75,Sq. M.,1,Qurum,Muscat
1801,Villa,75000,OMR,221,Sq. M.,3,Al Seeb,Muscat
1804,Villa,1000,OMR,1000,Sq. M.,4,Haima,Al Wusta
1805,Apartment,600,OMR,10,Sq. M.,1,Mahut,Al Wusta


## Read Open Sooq Data, Clean It, and Apply Feature Engineering


In [87]:
df_2 = pd.read_csv("souq_detailed.csv")
df_2

,prop_price,prop_title,prop_location,prop_size
0,"79,000 OMR",72 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Mouj",72 m2 Studio Apartments for Sale in Muscat Al ...
1,"8,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
2,"67,000 OMR",416 m2 More than 6 bedrooms Villa for Sale in ...,"Muscat, Amerat",416 m2 More than 6 bedrooms Villa for Sale in ...
3,"6,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
4,"43,000 OMR",92 m2 3 Bedrooms Apartments for Sale in Muscat...,"Muscat, Al Mawaleh",92 m2 3 Bedrooms Apartments for Sale in Muscat...
...,...,...,...,...
8262,"42,500 OMR",116 m2 2 Bedrooms Apartments for Sale in Musca...,"Muscat, Al Khoud",116 m2 2 Bedrooms Apartments for Sale in Musca...
8263,15 OMR,120 m2 2 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",120 m2 2 Bedrooms Apartments for Rent in Musca...
8264,"1,200 OMR",80 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Khoud",80 m2 Studio Apartments for Sale in Muscat Al ...
8265,200 OMR,100 m2 3 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",100 m2 3 Bedrooms Apartments for Rent in Musca...


In [88]:
df_2.isnull().sum()

prop_price       5
prop_title       0
prop_location    0
prop_size        0
dtype: int64

In [89]:
df_2[df_2['prop_price'].isna()]

,prop_price,prop_title,prop_location,prop_size
323,NaN,Residential Land for Sale in Al Sharqiya Other,"Al Sharqiya, Other",Residential Land for Sale in Al Sharqiya Other
951,NaN,Residential Land for Sale in Al Sharqiya Other,"Al Sharqiya, Other",Residential Land for Sale in Al Sharqiya Other
1587,NaN,300 m2 4 Bedrooms Villa for Sale in Muscat Amerat,"Muscat, Amerat",300 m2 4 Bedrooms Villa for Sale in Muscat Amerat
6114,NaN,Shop for rent and equipments for sale in the h...,"Muscat, Al Khoud",Shop for rent and equipments for sale in the h...
7973,NaN,بيع محل لبيع الملابس الجاهزة,"Muscat, Amerat",بيع محل لبيع الملابس الجاهزة


In [90]:
df_2.dropna(subset=['prop_price'], inplace=True)
df_2

,prop_price,prop_title,prop_location,prop_size
0,"79,000 OMR",72 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Mouj",72 m2 Studio Apartments for Sale in Muscat Al ...
1,"8,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
2,"67,000 OMR",416 m2 More than 6 bedrooms Villa for Sale in ...,"Muscat, Amerat",416 m2 More than 6 bedrooms Villa for Sale in ...
3,"6,500 OMR",Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
4,"43,000 OMR",92 m2 3 Bedrooms Apartments for Sale in Muscat...,"Muscat, Al Mawaleh",92 m2 3 Bedrooms Apartments for Sale in Muscat...
...,...,...,...,...
8262,"42,500 OMR",116 m2 2 Bedrooms Apartments for Sale in Musca...,"Muscat, Al Khoud",116 m2 2 Bedrooms Apartments for Sale in Musca...
8263,15 OMR,120 m2 2 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",120 m2 2 Bedrooms Apartments for Rent in Musca...
8264,"1,200 OMR",80 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Khoud",80 m2 Studio Apartments for Sale in Muscat Al ...
8265,200 OMR,100 m2 3 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",100 m2 3 Bedrooms Apartments for Rent in Musca...


In [91]:
df_2[df_2['prop_price'] == 'not available']

,prop_price,prop_title,prop_location,prop_size


In [92]:
df_2[df_2['prop_title'] == 'not available']

,prop_price,prop_title,prop_location,prop_size


In [93]:
df_2[df_2['prop_location'] == 'not available']

,prop_price,prop_title,prop_location,prop_size


In [94]:
df_2[df_2['prop_size'] == 'not available']

,prop_price,prop_title,prop_location,prop_size


In [95]:
df_2['prop_price'] = df_2['prop_price'].str.replace('OMR', '', regex=False).str.replace(',', '', regex=False).str.strip()
df_2['prop_price'] = pd.to_numeric(df_2['prop_price'], errors='coerce')
df_2.insert(loc=df_2.columns.get_loc('prop_price') + 1, column='prop_price_unit', value='OMR')
df_2

,prop_price,prop_price_unit,prop_title,prop_location,prop_size
0,79000,OMR,72 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Mouj",72 m2 Studio Apartments for Sale in Muscat Al ...
1,8500,OMR,Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
2,67000,OMR,416 m2 More than 6 bedrooms Villa for Sale in ...,"Muscat, Amerat",416 m2 More than 6 bedrooms Villa for Sale in ...
3,6500,OMR,Residential Land for Sale in Muscat Amerat,"Muscat, Amerat",Residential Land for Sale in Muscat Amerat
4,43000,OMR,92 m2 3 Bedrooms Apartments for Sale in Muscat...,"Muscat, Al Mawaleh",92 m2 3 Bedrooms Apartments for Sale in Muscat...
...,...,...,...,...,...
8262,42500,OMR,116 m2 2 Bedrooms Apartments for Sale in Musca...,"Muscat, Al Khoud",116 m2 2 Bedrooms Apartments for Sale in Musca...
8263,15,OMR,120 m2 2 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",120 m2 2 Bedrooms Apartments for Rent in Musca...
8264,1200,OMR,80 m2 Studio Apartments for Sale in Muscat Al ...,"Muscat, Al Khoud",80 m2 Studio Apartments for Sale in Muscat Al ...
8265,200,OMR,100 m2 3 Bedrooms Apartments for Rent in Musca...,"Muscat, Al Maabilah",100 m2 3 Bedrooms Apartments for Rent in Musca...


In [96]:
df_2[['governorate', 'state']] = df_2['prop_location'].str.split(',', n=1, expand=True)
df_2['governorate'] = df_2['governorate'].str.strip()
df_2['state'] = df_2['state'].str.strip()
df_2.drop(columns=['prop_location'], inplace=True)
df_2

,prop_price,prop_price_unit,prop_title,prop_size,governorate,state
0,79000,OMR,72 m2 Studio Apartments for Sale in Muscat Al ...,72 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Mouj
1,8500,OMR,Residential Land for Sale in Muscat Amerat,Residential Land for Sale in Muscat Amerat,Muscat,Amerat
2,67000,OMR,416 m2 More than 6 bedrooms Villa for Sale in ...,416 m2 More than 6 bedrooms Villa for Sale in ...,Muscat,Amerat
3,6500,OMR,Residential Land for Sale in Muscat Amerat,Residential Land for Sale in Muscat Amerat,Muscat,Amerat
4,43000,OMR,92 m2 3 Bedrooms Apartments for Sale in Muscat...,92 m2 3 Bedrooms Apartments for Sale in Muscat...,Muscat,Al Mawaleh
...,...,...,...,...,...,...
8262,42500,OMR,116 m2 2 Bedrooms Apartments for Sale in Musca...,116 m2 2 Bedrooms Apartments for Sale in Musca...,Muscat,Al Khoud
8263,15,OMR,120 m2 2 Bedrooms Apartments for Rent in Musca...,120 m2 2 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah
8264,1200,OMR,80 m2 Studio Apartments for Sale in Muscat Al ...,80 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Khoud
8265,200,OMR,100 m2 3 Bedrooms Apartments for Rent in Musca...,100 m2 3 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah


In [97]:
#Define the function to extract type
def extract_property_type(title):
    title = title.lower()
    if 'villa' in title:
        return 'Villa'
    elif 'apartment' in title or 'studio' in title:
        return 'Apartment'
    elif 'land' in title:
        return 'Land'
    elif 'commercial' in title:
        return 'Commercial'
    elif 'building' in title:
        return 'Building'
    elif 'office' in title:
        return 'Office'
    elif 'shop' in title:
        return 'Shop'
    else:
        return 'not defined'

# Apply the function to create new column
df_2['prop_title'] = df_2['prop_title'].apply(extract_property_type)

In [98]:
df_2

,prop_price,prop_price_unit,prop_title,prop_size,governorate,state
0,79000,OMR,Apartment,72 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Mouj
1,8500,OMR,Land,Residential Land for Sale in Muscat Amerat,Muscat,Amerat
2,67000,OMR,Villa,416 m2 More than 6 bedrooms Villa for Sale in ...,Muscat,Amerat
3,6500,OMR,Land,Residential Land for Sale in Muscat Amerat,Muscat,Amerat
4,43000,OMR,Apartment,92 m2 3 Bedrooms Apartments for Sale in Muscat...,Muscat,Al Mawaleh
...,...,...,...,...,...,...
8262,42500,OMR,Apartment,116 m2 2 Bedrooms Apartments for Sale in Musca...,Muscat,Al Khoud
8263,15,OMR,Apartment,120 m2 2 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah
8264,1200,OMR,Apartment,80 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Khoud
8265,200,OMR,Apartment,100 m2 3 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah


In [99]:
def extract_rooms_from_size(value):
    if pd.isna(value):
        return 'not available'
    
    value = str(value).lower()
    match = re.search(r'(\d+)\s*(bedroom|bedrooms|bhk)', value)
    if match:
        return int(match.group(1))
    else:
        return 'not available'

# Apply to new column
df_2['number_room'] = df_2['prop_size'].apply(extract_rooms_from_size)
df_2

,prop_price,prop_price_unit,prop_title,prop_size,governorate,state,number_room
0,79000,OMR,Apartment,72 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Mouj,not available
1,8500,OMR,Land,Residential Land for Sale in Muscat Amerat,Muscat,Amerat,not available
2,67000,OMR,Villa,416 m2 More than 6 bedrooms Villa for Sale in ...,Muscat,Amerat,6
3,6500,OMR,Land,Residential Land for Sale in Muscat Amerat,Muscat,Amerat,not available
4,43000,OMR,Apartment,92 m2 3 Bedrooms Apartments for Sale in Muscat...,Muscat,Al Mawaleh,3
...,...,...,...,...,...,...,...
8262,42500,OMR,Apartment,116 m2 2 Bedrooms Apartments for Sale in Musca...,Muscat,Al Khoud,2
8263,15,OMR,Apartment,120 m2 2 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah,2
8264,1200,OMR,Apartment,80 m2 Studio Apartments for Sale in Muscat Al ...,Muscat,Al Khoud,not available
8265,200,OMR,Apartment,100 m2 3 Bedrooms Apartments for Rent in Musca...,Muscat,Al Maabilah,3


In [100]:
def extract_size_and_unit(size_text):
    if pd.isna(size_text) or size_text.strip().lower() == 'not available':
        return 'not available', 'not available'
    
    size_text = size_text.lower()
    
    match = re.search(r'(\d+\.?\d*)\s*(m2|sqm|sq\.m|sqft|ft2|square meters|square feet)', size_text)
    if match:
        size = float(match.group(1))
        unit = match.group(2)

        # Convert to Sq. M. if needed
        if unit in ['sqft', 'ft2', 'square feet']:
            size = round(size * 0.092903, 2)
        elif unit in ['m2', 'sqm', 'sq.m', 'square meters']:
            size = round(size, 2)
        else:
            return 'not available', 'not available'

        return size, 'Sq. M.'
    else:
        return 'not available', 'not available'

# Apply the function
df_2[['prop_size', 'prop_size_unit']] = df_2['prop_size'].apply(lambda x: pd.Series(extract_size_and_unit(x)))
df_2

,prop_price,prop_price_unit,prop_title,prop_size,governorate,state,number_room,prop_size_unit
0,79000,OMR,Apartment,72.0,Muscat,Al Mouj,not available,Sq. M.
1,8500,OMR,Land,not available,Muscat,Amerat,not available,not available
2,67000,OMR,Villa,416.0,Muscat,Amerat,6,Sq. M.
3,6500,OMR,Land,not available,Muscat,Amerat,not available,not available
4,43000,OMR,Apartment,92.0,Muscat,Al Mawaleh,3,Sq. M.
...,...,...,...,...,...,...,...,...
8262,42500,OMR,Apartment,116.0,Muscat,Al Khoud,2,Sq. M.
8263,15,OMR,Apartment,120.0,Muscat,Al Maabilah,2,Sq. M.
8264,1200,OMR,Apartment,80.0,Muscat,Al Khoud,not available,Sq. M.
8265,200,OMR,Apartment,100.0,Muscat,Al Maabilah,3,Sq. M.


In [102]:
size_unit_col = df_2.pop('prop_size_unit')
df_2.insert(loc=df_2.columns.get_loc('prop_size') + 1, column='prop_size_units', value=size_unit_col)
df_2

,prop_price,prop_price_unit,prop_title,prop_size,prop_size_units,governorate,state,number_room
0,79000,OMR,Apartment,72.0,Sq. M.,Muscat,Al Mouj,not available
1,8500,OMR,Land,not available,not available,Muscat,Amerat,not available
2,67000,OMR,Villa,416.0,Sq. M.,Muscat,Amerat,6
3,6500,OMR,Land,not available,not available,Muscat,Amerat,not available
4,43000,OMR,Apartment,92.0,Sq. M.,Muscat,Al Mawaleh,3
...,...,...,...,...,...,...,...,...
8262,42500,OMR,Apartment,116.0,Sq. M.,Muscat,Al Khoud,2
8263,15,OMR,Apartment,120.0,Sq. M.,Muscat,Al Maabilah,2
8264,1200,OMR,Apartment,80.0,Sq. M.,Muscat,Al Khoud,not available
8265,200,OMR,Apartment,100.0,Sq. M.,Muscat,Al Maabilah,3


In [103]:
df_2.isnull().sum()

prop_price         0
prop_price_unit    0
prop_title         0
prop_size          0
prop_size_units    0
governorate        0
state              0
number_room        0
dtype: int64

In [104]:
df_2.dtypes

prop_price          int64
prop_price_unit    object
prop_title         object
prop_size          object
prop_size_units    object
governorate        object
state              object
number_room        object
dtype: object

In [105]:
df_2['prop_size'] = pd.to_numeric(df_2['prop_size'], errors='coerce')
df_2['number_room'] = pd.to_numeric(df_2['number_room'], errors='coerce')
df_2.dtypes

prop_price           int64
prop_price_unit     object
prop_title          object
prop_size          float64
prop_size_units     object
governorate         object
state               object
number_room        float64
dtype: object

## Combine Open Sooq and Bayut Cleaned Data 

In [106]:
df = pd.concat([df_1, df_2])
df

,prop_title,prop_price,prop_price_unit,prop_size,prop_size_units,number_room,state,governorate
0,Villa,1200,OMR,1500.0,Sq. M.,5.0,Al Buraimi,Al Buraimi
1,Villa,1500,OMR,1500.0,Sq. M.,5.0,Qantab,Muscat
2,Apartment,1200,OMR,1000.0,Sq. M.,3.0,Mahut,Al Wusta
3,Villa,1500,OMR,1000.0,Sq. M.,5.0,Al Sifah,Muscat
4,Villa,1000,OMR,1000.0,Sq. M.,4.0,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
8262,Apartment,42500,OMR,116.0,Sq. M.,2.0,Al Khoud,Muscat
8263,Apartment,15,OMR,120.0,Sq. M.,2.0,Al Maabilah,Muscat
8264,Apartment,1200,OMR,80.0,Sq. M.,NaN,Al Khoud,Muscat
8265,Apartment,200,OMR,100.0,Sq. M.,3.0,Al Maabilah,Muscat


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9482 entries, 0 to 8266
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   prop_title       9482 non-null   object 
 1   prop_price       9482 non-null   int64  
 2   prop_price_unit  9482 non-null   object 
 3   prop_size        3744 non-null   float64
 4   prop_size_units  9482 non-null   object 
 5   number_room      3895 non-null   float64
 6   state            9482 non-null   object 
 7   governorate      9480 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 666.7+ KB


In [108]:
df[df['governorate'].isna()]

,prop_title,prop_price,prop_price_unit,prop_size,prop_size_units,number_room,state,governorate
1260,Apartment,36000,OMR,82.0,Sq. M.,2.0,Dhofar,None
1395,Apartment,212000,OMR,212.0,Sq. M.,3.0,Muscat,None


In [109]:
df['governorate'] = df['governorate'].fillna(df['state'])
df

,prop_title,prop_price,prop_price_unit,prop_size,prop_size_units,number_room,state,governorate
0,Villa,1200,OMR,1500.0,Sq. M.,5.0,Al Buraimi,Al Buraimi
1,Villa,1500,OMR,1500.0,Sq. M.,5.0,Qantab,Muscat
2,Apartment,1200,OMR,1000.0,Sq. M.,3.0,Mahut,Al Wusta
3,Villa,1500,OMR,1000.0,Sq. M.,5.0,Al Sifah,Muscat
4,Villa,1000,OMR,1000.0,Sq. M.,4.0,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
8262,Apartment,42500,OMR,116.0,Sq. M.,2.0,Al Khoud,Muscat
8263,Apartment,15,OMR,120.0,Sq. M.,2.0,Al Maabilah,Muscat
8264,Apartment,1200,OMR,80.0,Sq. M.,NaN,Al Khoud,Muscat
8265,Apartment,200,OMR,100.0,Sq. M.,3.0,Al Maabilah,Muscat


In [110]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9482 entries, 0 to 8266
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   prop_title       9482 non-null   object 
 1   prop_price       9482 non-null   int64  
 2   prop_price_unit  9482 non-null   object 
 3   prop_size        3744 non-null   float64
 4   prop_size_units  9482 non-null   object 
 5   number_room      3895 non-null   float64
 6   state            9482 non-null   object 
 7   governorate      9482 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 666.7+ KB


In [111]:
df.fillna('not available', inplace=True)
df

C:\Users\bbuser\AppData\Local\Temp\ipykernel_24808\162090932.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'not available' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('not available', inplace=True)


,prop_title,prop_price,prop_price_unit,prop_size,prop_size_units,number_room,state,governorate
0,Villa,1200,OMR,1500.0,Sq. M.,5.0,Al Buraimi,Al Buraimi
1,Villa,1500,OMR,1500.0,Sq. M.,5.0,Qantab,Muscat
2,Apartment,1200,OMR,1000.0,Sq. M.,3.0,Mahut,Al Wusta
3,Villa,1500,OMR,1000.0,Sq. M.,5.0,Al Sifah,Muscat
4,Villa,1000,OMR,1000.0,Sq. M.,4.0,Hamra,Al Dakhiliya
...,...,...,...,...,...,...,...,...
8262,Apartment,42500,OMR,116.0,Sq. M.,2.0,Al Khoud,Muscat
8263,Apartment,15,OMR,120.0,Sq. M.,2.0,Al Maabilah,Muscat
8264,Apartment,1200,OMR,80.0,Sq. M.,not available,Al Khoud,Muscat
8265,Apartment,200,OMR,100.0,Sq. M.,3.0,Al Maabilah,Muscat


In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9482 entries, 0 to 8266
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   prop_title       9482 non-null   object
 1   prop_price       9482 non-null   int64 
 2   prop_price_unit  9482 non-null   object
 3   prop_size        9482 non-null   object
 4   prop_size_units  9482 non-null   object
 5   number_room      9482 non-null   object
 6   state            9482 non-null   object
 7   governorate      9482 non-null   object
dtypes: int64(1), object(7)
memory usage: 666.7+ KB
